Install libraries

In [ ]:
%pip install requests
%pip install pandas
%pip install numpy
%pip install scikit-learn
%pip install statstools
%pip install matplotlib

In [ ]:
import requests
import pandas as pd
import numpy as np
from sklearn.cluster import DBSCAN

Get hourly weather data with ETL framework

In [ ]:
parameters = {"ids": "KSBA",
              "format": "json",
              "taf": "false",
              "hours": 1000}
url = "https://aviationweather.gov/api/data/metar"
data = requests.get(url, params = parameters).json()
df = pd.json_normalize(data)
df.head()

Convert observation time and temperature into UTC and Fahrenheit units

In [ ]:
df["UTC"] = pd.to_datetime(df["obsTime"], unit = "s")
df["Temperature"] = np.round(np.array(df["temp"])*1.8 + 32, 0)
df = df.drop(["clouds","obsTime","temp"], axis = 1)
df.head()

In [ ]:
x = np.array(df["Temperature"]).reshape(-1, 1)

In [ ]:
clusters = DBSCAN(eps=1, min_samples=12).fit_predict(x)
print(clusters)

In [ ]:
df = df[["UTC","Temperature"]].assign(Anomaly = clusters)

In [ ]:
df.head()

In [ ]:
df.loc[(df["Anomaly"] == -1),:]

# Time Series Decomposition